In [1]:
import sys
from utils import NotebookFinder
sys.meta_path.append(NotebookFinder())

import os
import numpy as np
import pandas as pd
from LogisticRegression import LogisticRegression
from ClassificationEvaluator import ClassificationEvaluator

# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier

#from Optimizers import Optimizers

from IPython.display import display, Markdown

import warnings
warnings.filterwarnings('ignore')

data_directory='../datasets/preprocessed'

C:\Users\agama\anaconda3\envs\qed\lib\site-packages\nbformat\current.py:15: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  warnings.warn("""nbformat.current is deprecated.


importing Jupyter notebook from LogisticRegression.ipynb


### Iterative-Reweighted Least Squares


Fitting stopped after 100 iterations
Time eclapsed for fitting: 2.6677470207214355 secs
Initial cost  0.6931471805599453
Final cost  0.43978836808899546





### Gradient Descent


...

ClassificationEvaluator class instance recall score: 1.0
,sklearn method result: 1.0

ClassificationEvaluator class instance recall score: 1.0
,sklearn method result: 1.0

### F1 score

ClassificationEvaluator class instance precision score: 0.7499999999999999
,sklearn method result: 0.7499999999999999

ClassificationEvaluator class instance precision score: 0.7499999999999999
,sklearn method result: 0.7499999999999999

## Data loading

In [2]:
frames={}
for filename in os.listdir(data_directory):
    # display(Markdown(f'### {filename} data'))
    data=pd.read_csv(f"{data_directory}/{filename}")
    # display(data.head(3))
    frames[filename.split('.')[0]]=np.array(data)
    
keys_lst=list(frames.keys())
subsets_lst=[tuple(keys_lst[i:i + 4]) for i in range(0, len(keys_lst), 4)]

## Experiments setup

In [18]:
# DEFAULT
b1=0.9
b2=0.999
eps=1e-8
sample_size = 1
learning_rate = 1e-4

### Convergence analysis: how the value of log-likelihood function depends on the number of iterations?

In [19]:
patience=None
min_delta=None
iterations = 1000 # [1, 2, 3, 4, 5, 10]  # TEMPORARY: TO BE SET

results={}
logreg = LogisticRegression()

In [20]:
for item in subsets_lst:
    
    # irls_cost_history, gd_cost_history, sgd_cost_history, adam_cost_history = [], [], [], []
    
    X_test_key, y_test_key, X_train_key, y_train_key = item
    X_test, y_test, X_train, y_train = frames[X_test_key], frames[y_test_key], frames[X_train_key], frames[y_train_key]
    X_train_b = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_test_b = np.c_[np.ones((X_test.shape[0], 1)), X_test]
    
    dataset_name=X_train_key.split("_")[0]
    
    #for iter_cnt in iterations:
    irls_kwds={"iterations": iterations, "min_delta":min_delta, "patience":patience}
    gd_kwds={"iterations": iterations, "alpha": learning_rate, "min_delta":min_delta, "patience":patience}
    sgd_kwds={"iterations": iterations, "alpha": learning_rate, "sample_size": sample_size,
        "min_delta":min_delta,"patience":patience}
    adam_kwds={"iterations": iterations, "b1": b1, "b2": b2, "alpha": learning_rate, "epsilon": eps,
        "min_delta":min_delta, "patience":patience}
    
    if dataset_name != 'adult':
        _,_,_,irls_cost_history=logreg.fit(X_train_b,y_train,**irls_kwds)
    _,_,_,gd_cost_history=logreg.fit(X_train_b,y_train,**gd_kwds)
    _,_,_,sgd_cost_history=logreg.fit(X_train_b,y_train,**sgd_kwds)
    _,_,_,adam_cost_history=logreg.fit(X_train,y_train,**adam_kwds)
    
        # irls_cost_history.append(irls_final_cost)
        # gd_cost_history.append(gd_final_cost)
        # sgd_cost_history.append(sgd_final_cost)
        # adam_cost_history.append(adam_final_cost)
    if dataset_name != 'adult':
        results[f"irls_{dataset_name}"]=irls_cost_history
    results[f"gd_{dataset_name}"]=gd_cost_history
    results[f"sgd_{dataset_name}"]=sgd_cost_history
    results[f"adam_{dataset_name}"]=adam_cost_history

results = pd.DataFrame(results)
results.index.name = 'n_iter'
results.to_csv('./../results/convergence_analysis_results.csv')

In [21]:
pd.DataFrame(results)

,gd_adult,sgd_adult,adam_adult,irls_credit,gd_credit,sgd_credit,adam_credit,irls_sick,gd_sick,sgd_sick,adam_sick,irls_titanic,gd_titanic,sgd_titanic,adam_titanic
n_iter,,,,,,,,,,,,,,,
0,0.992724,0.628393,2.114196,0.465079,1.398631,1.419481,1.208808,0.231028,0.653822,1.347334,1.370283,0.175332,1.318263,0.884398,1.678512
1,0.588912,0.529789,2.112350,0.441843,1.388631,1.406913,1.207982,0.150539,0.588873,1.159777,1.369237,0.104728,1.240245,0.859623,1.677243
2,0.511509,0.476491,2.110200,0.439815,1.379047,1.394723,1.207023,0.116205,0.535844,0.999900,1.368019,0.081424,1.170369,0.835734,1.675766
3,0.466185,0.444419,2.107838,0.439788,1.369860,1.382884,1.205972,0.104163,0.492305,0.865417,1.366681,0.072949,1.107863,0.812698,1.674143
4,0.437439,0.423924,2.105323,0.439788,1.361049,1.371380,1.204857,0.101150,0.456322,0.753584,1.365257,0.070480,1.051910,0.790484,1.672416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.325288,0.325125,1.383436,0.439788,0.483992,0.473605,0.954375,0.100736,0.105210,0.107822,0.944143,0.070028,0.086486,0.089544,1.166573
996,0.325287,0.325122,1.383060,0.439788,0.483921,0.473560,0.954251,0.100736,0.105207,0.107818,0.943910,0.070028,0.086478,0.089535,1.166288
997,0.325286,0.325122,1.382684,0.439788,0.483850,0.473514,0.954126,0.100736,0.105204,0.107813,0.943678,0.070028,0.086470,0.089525,1.166004


# 2.1 Learning rate evaluation

In [53]:
b1=0.9
b2=0.999
eps=1e-8
sample_size = 1
learning_rate = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]

patience=None
min_delta=None
iterations = 1000 # TEMPORARY: TO BE SET

results={}
logreg = LogisticRegression()

In [54]:
for item in subsets_lst:
    
    results = {}
    gd_evaluation_acc, sgd_evaluation_acc, adam_evaluation_acc = [], [], []
    gd_evaluation_f1, sgd_evaluation_f1, adam_evaluation_f1 = [], [], []
    cost_results = {}
    gd_history, sgd_history, adam_history = [], [], []

    X_test_key, y_test_key, X_train_key, y_train_key = item
    X_test, y_test, X_train, y_train = frames[X_test_key], frames[y_test_key], frames[X_train_key], frames[y_train_key]
    X_train_b = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_test_b = np.c_[np.ones((X_test.shape[0], 1)), X_test]
    y_test = [y for row in y_test for y in row]

    dataset_name=X_train_key.split("_")[0]
    
    for lr in learning_rate:
        gd_kwds={"iterations": iterations, "alpha": lr, "min_delta":min_delta, "patience":patience}
        sgd_kwds={"iterations": iterations, "alpha": lr, "sample_size": sample_size,
            "min_delta":min_delta,"patience":patience}
        adam_kwds={"iterations": iterations, "b1": b1, "b2": b2, "alpha": lr, "epsilon": eps,
            "min_delta":min_delta, "patience":patience}

        gd_params, _, _, gd_cost_history = logreg.fit(X_train_b,y_train,**gd_kwds)
        _, gd_pred = logreg.predict(X_test_b, gd_params)
        gd_metrics_acc = ClassificationEvaluator().accuracy(y_test,gd_pred)
        gd_metrics_f1 = ClassificationEvaluator().f1_score(y_test,gd_pred)

        sgd_params, _, _, sgd_cost_history = logreg.fit(X_train_b,y_train,**sgd_kwds)
        _, sgd_pred = logreg.predict(X_test_b, sgd_params)
        sgd_metrics_acc = ClassificationEvaluator().accuracy(y_test,sgd_pred)
        sgd_metrics_f1 = ClassificationEvaluator().f1_score(y_test,sgd_pred)

        adam_params, _, _, adam_cost_history = logreg.fit(X_train,y_train,**adam_kwds)
        _, adam_pred = logreg.predict(X_test, adam_params[0], adam_params[1])
        adam_metrics_acc = ClassificationEvaluator().accuracy(y_test,adam_pred)
        adam_metrics_f1 = ClassificationEvaluator().f1_score(y_test,adam_pred)
        
        gd_evaluation_acc.append(gd_metrics_acc)
        sgd_evaluation_acc.append(sgd_metrics_acc)
        adam_evaluation_acc.append(adam_metrics_acc)

        gd_evaluation_f1.append(gd_metrics_f1)
        sgd_evaluation_f1.append(sgd_metrics_f1)
        adam_evaluation_f1.append(adam_metrics_f1)

        gd_history.append(gd_cost_history)
        sgd_history.append(sgd_cost_history)
        adam_history.append(adam_cost_history)

    results_acc = pd.concat([pd.Series(gd_evaluation_acc), pd.Series(sgd_evaluation_acc), pd.Series(adam_evaluation_acc)], axis=1)
    results_acc.columns = ['gd', 'sgd', 'adam']
    results_acc.index = learning_rate
    results_acc.index.name = "Learning rate"
    display(results_acc)
    results_acc_melted = pd.melt(results.reset_index(), id_vars=['Learning rate'], value_vars=['gd', 'sgd', 'adam'])
    results_acc_melted.columns=['Learning rate', 'Algorithm', 'Accuracy']

    results_acc_melted.to_csv(f"./../results/learning_rate_acc_{dataset_name}.csv")

    results_f1 = pd.concat([pd.Series(gd_evaluation_f1), pd.Series(sgd_evaluation_f1), pd.Series(adam_evaluation_f1)], axis=1)
    results_f1.columns = ['gd', 'sgd', 'adam']
    results_f1.index = learning_rate
    results_f1.index.name = "Learning rate"
    display(results_f1)
    results_f1_melted = pd.melt(results.reset_index(), id_vars=['Learning rate'], value_vars=['gd', 'sgd', 'adam'])
    results_f1_melted.columns=['Learning rate', 'Algorithm', 'F1-Score']

    results_f1_melted.to_csv(f"./../results/learning_rate_f1_{dataset_name}.csv")

    cost_results = pd.concat([pd.DataFrame(gd_history), pd.DataFrame(sgd_history), pd.DataFrame(adam_history)], axis=1)
    columns = []
    for alg in ['gd', 'sgd', 'adam']:
        for l in learning_rate:
            l_e = '{:.0e}'.format(l)
            columns.append(alg+f"_{l_e}")
    cost_results.columns = columns
    cost_results.index.name = 'n_iter'
    cost_results.to_csv(f'./../results/learning_rate_cost_{dataset_name}.csv')

# 2.2 ADAM b1/b2 evaluation

In [48]:
b1s=[0.5, 0.6, 0.7, 0.8, 0.9, 0.999]
b2s=[0.5, 0.6, 0.7, 0.8, 0.9, 0.999]
eps=1e-8 #[1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
sample_size = 1
learning_rate = 1e-4

patience=None
min_delta=None
iterations = 1000 # TEMPORARY: TO BE SET

results=pd.DataFrame()
logreg = LogisticRegression()

In [49]:
for item in subsets_lst:
    
    adam_evaluation = []
    
    X_test_key, y_test_key, X_train_key, y_train_key = item
    X_test, y_test, X_train, y_train = frames[X_test_key], frames[y_test_key], frames[X_train_key], frames[y_train_key]
    X_train_b = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_test_b = np.c_[np.ones((X_test.shape[0], 1)), X_test]
    y_test = [y for row in y_test for y in row]

    dataset_name=X_train_key.split("_")[0]
    
    for b1 in b1s:
        adam_b1_evaluation = []
        for b2 in b2s:
            adam_kwds={"iterations": iterations, "b1": b1, "b2": b2, "alpha": lr, "epsilon": eps,
            "min_delta":min_delta, "patience":patience}

            adam_params, _, _, _ = logreg.fit(X_train,y_train,**adam_kwds)
            _, adam_pred = logreg.predict(X_test, adam_params[0], adam_params[1])
            adam_metrics = ClassificationEvaluator().f1_score(y_test,adam_pred)

            adam_b1_evaluation.append(adam_metrics)
        adam_evaluation.append(adam_b1_evaluation)

    results = pd.DataFrame(adam_evaluation)
    results.columns = b2s
    results.index = b1s
    results.index.name = "b1"
    display(results)

    results.to_csv(f"./../results/adam_b1_b2_{dataset_name}.csv")

,0.500,0.600,0.700,0.800,0.900,0.999
b1,,,,,,
0.500,0.664386,0.661303,0.666163,0.665532,0.662722,0.665529
0.600,0.664386,0.664765,0.662469,0.664002,0.664639,0.665655
0.700,0.663624,0.665148,0.665402,0.665657,0.665152,0.665402
0.800,0.664767,0.664894,0.665149,0.665151,0.665024,0.665403
0.900,0.664765,0.664896,0.664768,0.664514,0.665024,0.665276
0.999,0.645253,0.633279,0.669426,0.672894,0.516267,0.635914


,0.500,0.600,0.700,0.800,0.900,0.999
b1,,,,,,
0.500,0.549296,0.549296,0.559441,0.553191,0.549296,0.549296
0.600,0.559441,0.557143,0.549296,0.549296,0.557143,0.549296
0.700,0.549296,0.557143,0.549296,0.557143,0.549296,0.549296
0.800,0.549296,0.557143,0.553191,0.553191,0.549296,0.549296
0.900,0.557143,0.553191,0.553191,0.553191,0.549296,0.549296
0.999,0.542553,0.566845,0.527027,0.427350,0.396552,0.535088


,0.500,0.600,0.700,0.800,0.900,0.999
b1,,,,,,
0.500,0.734694,0.742268,0.742268,0.742268,0.742268,0.742268
0.600,0.734694,0.734694,0.742268,0.742268,0.742268,0.742268
0.700,0.742268,0.734694,0.734694,0.742268,0.742268,0.742268
0.800,0.742268,0.734694,0.734694,0.742268,0.734694,0.755102
0.900,0.734694,0.734694,0.742268,0.742268,0.742268,0.734694
0.999,0.144928,0.184615,0.520000,0.666667,0.733945,0.709091


,0.500,0.600,0.700,0.800,0.900,0.999
b1,,,,,,
0.500,0.954733,0.954733,0.954733,0.954733,0.954733,0.954733
0.600,0.954733,0.954733,0.954733,0.954733,0.954733,0.954733
0.700,0.954733,0.954733,0.954733,0.954733,0.954733,0.954733
0.800,0.954733,0.954733,0.954733,0.954733,0.954733,0.954733
0.900,0.954733,0.954733,0.954733,0.954733,0.954733,0.954733
0.999,0.942149,0.829694,0.886275,0.944000,0.879121,0.936508


# 2.3 ADAM epsilon evaluation

In [ ]:
# TODO

# 4 - comparison with LDA/QDA/KNN

In [50]:
b1=0.9
b2=0.999
eps=1e-8
sample_size = 1
learning_rate = 1e-4 # na tym sie wywala gd (czasem)

patience=None # x razy bez zmian wiekszych niz min_delta
min_delta=None # 
iterations = 1000  # 1000
# co ze zbieganiem po 1000 iteracjach/early stoppingiem

logreg = LogisticRegression()

In [51]:
lda = LDA()
qda = QDA()
knn = KNeighborsClassifier(n_neighbors=20)

In [52]:
for item in subsets_lst:
    
    X_test_key, y_test_key, X_train_key, y_train_key = item
    X_test, y_test, X_train, y_train = frames[X_test_key], frames[y_test_key], frames[X_train_key], frames[y_train_key]
    X_train_b = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_test_b = np.c_[np.ones((X_test.shape[0], 1)), X_test]
    y_test = [y for row in y_test for y in row]

    dataset_name=X_train_key.split("_")[0]
    
    irls_kwds={"iterations": iterations, "min_delta":min_delta, "patience":patience}
    gd_kwds={"iterations": iterations, "alpha": learning_rate, "min_delta":min_delta, "patience":patience}
    sgd_kwds={"iterations": iterations, "alpha": learning_rate, "sample_size": sample_size,
        "min_delta":min_delta,"patience":patience}
    adam_kwds={"iterations": iterations, "b1": b1, "b2": b2, "alpha": learning_rate, "epsilon": eps,
        "min_delta":min_delta, "patience":patience}
    
    if dataset_name != 'adult':
        irls_params, _, _, _ = logreg.fit(X_train_b,y_train,**irls_kwds)
        _, irls_pred = logreg.predict(X_test_b, irls_params)
        irls_metrics = ClassificationEvaluator().calculate_all(y_test,irls_pred)

    gd_params, _, _, _ = logreg.fit(X_train_b,y_train,**gd_kwds)
    _, gd_pred = logreg.predict(X_test_b, gd_params)
    gd_metrics = ClassificationEvaluator().calculate_all(y_test,gd_pred)

    sgd_params, _, _, _ = logreg.fit(X_train_b,y_train,**sgd_kwds)
    _, sgd_pred = logreg.predict(X_test_b, sgd_params)
    sgd_metrics = ClassificationEvaluator().calculate_all(y_test,sgd_pred)

    adam_params, _, _, _ = logreg.fit(X_train,y_train,**adam_kwds)
    _, adam_pred = logreg.predict(X_test, adam_params[0], adam_params[1])
    adam_metrics = ClassificationEvaluator().calculate_all(y_test,adam_pred)

    lda.fit(X_train, y_train)
    lda_pred = lda.predict(X_test)
    lda_metrics = ClassificationEvaluator().calculate_all(y_test,lda_pred)

    qda.fit(X_train, y_train)
    qda_pred = qda.predict(X_test)
    qda_metrics = ClassificationEvaluator().calculate_all(y_test,qda_pred)

    knn.fit(X_train, y_train)
    knn_pred = knn.predict(X_test)
    knn_metrics = ClassificationEvaluator().calculate_all(y_test,knn_pred)

    index=['LR - IRLS', 'LR - GD', 'LR - SGD', 'LR - ADAM', 'LDA', 'QDA', 'KNN']
    if dataset_name == 'adult':
        results = pd.DataFrame([gd_metrics, sgd_metrics, adam_metrics, lda_metrics, qda_metrics, knn_metrics]) 
        index.pop(0)
    else:
        results = pd.DataFrame([irls_metrics, gd_metrics, sgd_metrics, adam_metrics, lda_metrics, qda_metrics, knn_metrics]) 
    results.index=index
    results.columns=['Accuracy', 'Precision', 'Recall', 'F1-Score']
    results = results.transpose()
    results.index.name="Metric"
    display(results)

    results.to_csv(f"./../results/comparison_with_popular_{dataset_name}.csv")

,LR - GD,LR - SGD,LR - ADAM,LDA,QDA,KNN
Metric,,,,,,
Accuracy,0.846308,0.845873,0.571540,0.835087,0.768113,0.845090
Precision,0.725052,0.724295,0.226505,0.698426,0.521983,0.724008
Recall,0.613525,0.612123,0.300631,0.590750,0.782060,0.607568
F1-Score,0.664642,0.663502,0.258356,0.640091,0.626087,0.660697


,LR - IRLS,LR - GD,LR - SGD,LR - ADAM,LDA,QDA,KNN
Metric,,,,,,,
Accuracy,0.744000,0.740000,0.744000,0.640000,0.728000,0.632000,0.736000
Precision,0.582090,0.580645,0.582090,0.368421,0.552239,0.417476,0.764706
Recall,0.520000,0.480000,0.520000,0.280000,0.493333,0.573333,0.173333
F1-Score,0.549296,0.525547,0.549296,0.318182,0.521127,0.483146,0.282609


,LR - IRLS,LR - GD,LR - SGD,LR - ADAM,LDA,QDA,KNN
Metric,,,,,,,
Accuracy,0.971983,0.970905,0.969828,0.778017,0.963362,0.233836,0.964440
Precision,0.900000,0.918919,0.875000,0.037500,0.852941,0.072084,1.000000
Recall,0.620690,0.586207,0.603448,0.103448,0.500000,0.948276,0.431034
F1-Score,0.734694,0.715789,0.714286,0.055046,0.630435,0.133983,0.602410


,LR - IRLS,LR - GD,LR - SGD,LR - ADAM,LDA,QDA,KNN
Metric,,,,,,,
Accuracy,0.966463,0.969512,0.969512,0.353659,0.969512,0.926829,0.957317
Precision,0.983051,0.983193,0.983193,0.330739,0.983193,0.874074,0.974359
Recall,0.928000,0.936000,0.936000,0.680000,0.936000,0.944000,0.912000
F1-Score,0.954733,0.959016,0.959016,0.445026,0.959016,0.907692,0.942149
